# Cleaning and Exploring eBay Cars Sales Data

In this guided project, we'll work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

The dataset was originally scraped and uploaded to Kaggle by user *orgesleka* but unfortunately the original dataset isn't available on Kaggle anymore, but you can find it here https://data.world/data-society/used-cars-data

A few modifications from the original dataset were made :  
- Sampled 50,000 data points from the full dataset, to ensure your code runs quickly in our hosted environment
- Dirtied the dataset a bit to more closely resemble what you would expect from a scraped dataset (the version uploaded to Kaggle was cleaned to be easier to work with)

The aim of this project is to : 
- **Clean the data**
- **Perform a basic analyze** of the included used car listings

In [1]:
#Import necessary librairies
import numpy as np
import pandas as pd
import operator
from collections import OrderedDict

In [2]:
#Read CSV file - UTF-8 encoding not working
autos = pd.read_csv('autos.csv', encoding="Latin-1")

In [3]:
#Display basic dataframe information
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

In [4]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Several observations can be made after looking at the data :  
- **NaN** values existing in 5 of the columns but none have more than ~20% null values
- Most of the columns are of string type
- For some columns, string data will be difficult to manipulate in its current state
     - `Price` column for example mixes string and numeric values
     - Timestamped columns mix dates with times
     - Column names in *camelcase* would be better in *snakecase*
     
In the next code blocks we will clean column names and string data.

## Cleaning Column Names and String Data

### Improve Column Names Clarity

*Camelcase* column names make the labels difficult to read and in some cases, lack clarity. Switching to *snakecase* will make the names more descriptive.  
For example going from `yearOfRegistration` to `registration_year`

In [5]:
#Print array of columns names
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [6]:
#Modify column names
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen']

### Deal with Data Inconsistency 

In [7]:
#Display descriptive statistics for all column (numeric and string)
autos.describe(include="all")

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-09 11:54:38,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


A few things are of note here :  
First of all, we can observe that there are a number of text columns where almost all of the values seem to be identical (unique almost equal to count).
It also would be best to convert the price and odometer columns to numeric types and renamed odometer to odometer_km.

In [8]:
print(autos['seller'].value_counts())
print(autos['offer_type'].value_counts())

privat        49999
gewerblich        1
Name: seller, dtype: int64
Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64


As we supposed before, columns `seller` and `offer_type` do not provide useful information as all the cells but one contain the exact same data. We will drop those columns next as well as turning the `price` and the `odometer` values to numeric ones.  
It would also be interesting to check the value range for those columns to verify if there isn't any unrealistic outliers.

In [9]:
#Drop columns
autos.drop(['seller', 'offer_type'], axis=1, inplace=True)
#Remove $ and , sign using vectorized method str.replace
autos['price'] = autos['price'].str.replace('$','')
autos['price'] = autos['price'].str.replace(',','')
#Remove km and , sting using vectorized method str.replace
autos['odometer'] = autos['odometer'].str.replace('km','')
autos['odometer'] = autos['odometer'].str.replace(',','')

In [10]:
#Convert columns as numeric types
autos['price'] = autos["price"].astype(float)
autos['odometer'] = autos["odometer"].astype(float)

In [11]:
#Rename odometer column
autos.rename({'odometer':'odometer_km'}, axis='columns', inplace=True)

### Price and Mileage - Consistency Check

In [12]:
#How many unique values
print("Price unique values :", autos['price'].unique().shape[0])
print("Odometer unique values :", autos['odometer_km'].unique().shape[0])

Price unique values : 2357
Odometer unique values : 13


In [13]:
#Check descriptive statistics for odometer columns
autos['odometer_km'].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [14]:
#Check descriptive statistics for odometer columns
autos['odometer_km'].value_counts().sort_index(ascending=False)

150000.0    32424
125000.0     5170
100000.0     2169
90000.0      1757
80000.0      1436
70000.0      1230
60000.0      1164
50000.0      1027
40000.0       819
30000.0       789
20000.0       784
10000.0       264
5000.0        967
Name: odometer_km, dtype: int64

Odometer values seem consistent, no need to further clean the data.

In [15]:
#Check descriptive statistics for price columns
autos['price'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [16]:
#Check descriptive statistics for price columns
autos['price'].value_counts().sort_index(ascending=False)

99999999.0       1
27322222.0       1
12345678.0       3
11111111.0       2
10000000.0       1
              ... 
5.0              2
3.0              1
2.0              3
1.0            156
0.0           1421
Name: price, Length: 2357, dtype: int64

For price values on the other and, it is clear that some prices are unrealistic (more than a million $ or less than 500 is already strange even more when the car make isn't a luxury one or complete lemon).  
Based on those observations, let's remove values **over \$500k** (14 datapoints) and **under \$500**)

In [17]:
#Boolean filter with limits as 500 and 500000
autos = autos[autos['price'].between(500,500000)]

In [18]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45097 entries, 0 to 49999
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date_crawled        45097 non-null  object 
 1   name                45097 non-null  object 
 2   price               45097 non-null  float64
 3   abtest              45097 non-null  object 
 4   vehicle_type        41422 non-null  object 
 5   registration_year   45097 non-null  int64  
 6   gearbox             43283 non-null  object 
 7   power_ps            45097 non-null  int64  
 8   model               43032 non-null  object 
 9   odometer_km         45097 non-null  float64
 10  registration_month  45097 non-null  int64  
 11  fuel_type           41898 non-null  object 
 12  brand               45097 non-null  object 
 13  unrepaired_damage   37580 non-null  object 
 14  ad_created          45097 non-null  object 
 15  nr_of_pictures      45097 non-null  int64  
 16  post

In [19]:
autos.describe(include="all")

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,45097,45097,45097.000000,45097,41422,45097.000000,43283,45097.000000,43032,45097.000000,45097.000000,41898,45097,37580,45097,45097.0,45097.000000,45097
unique,43660,34575,NaN,2,8,NaN,2,NaN,244,NaN,NaN,7,40,2,76,NaN,NaN,35960
top,2016-03-29 23:42:13,BMW_316i,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,75,NaN,23266,12069,NaN,33364,NaN,3622,NaN,NaN,26909,9634,33906,1761,NaN,NaN,8
mean,NaN,NaN,6320.659600,NaN,NaN,2005.064173,NaN,121.559372,NaN,125293.035013,5.907089,NaN,NaN,NaN,NaN,0.0,51290.959798,NaN
std,NaN,NaN,9261.841444,NaN,NaN,89.652017,NaN,205.728640,NaN,39622.744927,3.632483,NaN,NaN,NaN,NaN,0.0,25700.453523,NaN
min,NaN,NaN,500.000000,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,1500.000000,NaN,NaN,2000.000000,NaN,75.000000,NaN,100000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30966.000000,NaN
50%,NaN,NaN,3500.000000,NaN,NaN,2004.000000,NaN,110.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,50226.000000,NaN
75%,NaN,NaN,7900.000000,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,72160.000000,NaN


After our first cleaning tasks :   
- There are still **NaN values** in some columns
- Timestamped columns are still unclear (mix of date and time) and need to be cleaned.

### Better Keep Track of Time

In [20]:
#Understand date range of the dates
#Use percentage inplace of count with normalize attribute and include NaN
#date_crawled has been added by the crawler
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025567
2016-03-06    0.014125
2016-03-07    0.036189
2016-03-08    0.033173
2016-03-09    0.032907
2016-03-10    0.032707
2016-03-11    0.033018
2016-03-12    0.037320
2016-03-13    0.015522
2016-03-14    0.036300
2016-03-15    0.034016
2016-03-16    0.029359
2016-03-17    0.031155
2016-03-18    0.012883
2016-03-19    0.034747
2016-03-20    0.038073
2016-03-21    0.037741
2016-03-22    0.033018
2016-03-23    0.032397
2016-03-24    0.028982
2016-03-25    0.031089
2016-03-26    0.032641
2016-03-27    0.031177
2016-03-28    0.034836
2016-03-29    0.033262
2016-03-30    0.033328
2016-03-31    0.031665
2016-04-01    0.033905
2016-04-02    0.035767
2016-04-03    0.038827
2016-04-04    0.036610
2016-04-05    0.013172
2016-04-06    0.003171
2016-04-07    0.001353
Name: date_crawled, dtype: float64

The data being crawled data we can, with the column studied in the previous block, confirm that the crawler did its job over a month (March 2016 to early April 2016), acquiring the data every day.

In [21]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2015-06-11    0.000022
2015-08-10    0.000022
2015-09-09    0.000022
2015-11-10    0.000022
2015-12-05    0.000022
                ...   
2016-04-03    0.039049
2016-04-04    0.036987
2016-04-05    0.011908
2016-04-06    0.003260
2016-04-07    0.001197
Name: ad_created, Length: 76, dtype: float64

All the adds included in the data set have been created between June 2015 and April 2016 even though most of the adds date back to **March 2016**.

In [22]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001087
2016-03-06    0.004169
2016-03-07    0.005211
2016-03-08    0.007007
2016-03-09    0.009468
2016-03-10    0.010289
2016-03-11    0.012041
2016-03-12    0.023904
2016-03-13    0.008870
2016-03-14    0.012285
2016-03-15    0.015677
2016-03-16    0.016165
2016-03-17    0.027674
2016-03-18    0.007406
2016-03-19    0.015411
2016-03-20    0.020423
2016-03-21    0.020667
2016-03-22    0.021243
2016-03-23    0.018405
2016-03-24    0.019536
2016-03-25    0.018582
2016-03-26    0.016476
2016-03-27    0.015456
2016-03-28    0.020534
2016-03-29    0.021354
2016-03-30    0.024148
2016-03-31    0.023438
2016-04-01    0.022862
2016-04-02    0.024880
2016-04-03    0.024946
2016-04-04    0.024303
2016-04-05    0.126616
2016-04-06    0.225314
2016-04-07    0.134155
Name: last_seen, dtype: float64

In [23]:
autos['registration_year'].describe()

count    45097.000000
mean      2005.064173
std         89.652017
min       1000.000000
25%       2000.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Something is wrong with years of registration.  
- Years cannot go beyond 2016 (as the crawled year is *2016*)
- Some years seem to be impossible as they go way back in time (*1900* would be a fair limit as to the first commercialized cars trace back to this time) 

In [24]:
#Filter on registration date
autos = autos[autos['registration_year'].between(1900,2016)]

In [25]:
autos['registration_year'].value_counts(normalize=True).sort_index()

1910    0.000046
1927    0.000023
1929    0.000023
1931    0.000023
1934    0.000046
          ...   
2012    0.030169
2013    0.018397
2014    0.015050
2015    0.008402
2016    0.021928
Name: registration_year, Length: 78, dtype: float64

The unrealistic years have been removed of the analysis. Most of the registration year of the vehicles in the adds is after the 90's

## Brand Making the Price ?

In [26]:
#Keep top 10 brands (ones with the most adds)
brands = autos['brand'].value_counts().head(10).index

In [27]:
#Create empty dict
price_wrt_brand = {}

for brand in brands:
    #Filter dataframe by brands
    selected_rows = autos[autos['brand'] == brand]
    #Comupute rounded mean selling price
    mean_price = round(selected_rows['price'].mean())
    #Append to dictionary
    price_wrt_brand[brand] = mean_price

In [28]:
#Sort dictionary with OrderedDict
sorted_tuples = sorted(price_wrt_brand.items(), key=operator.itemgetter(1), 
                      reverse=True)

sorted_dict = OrderedDict()
for k, v in sorted_tuples:
    sorted_dict[k] = v

In [29]:
sorted_dict

OrderedDict([('audi', 9614),
             ('mercedes_benz', 8767),
             ('bmw', 8582),
             ('volkswagen', 5784),
             ('seat', 4811),
             ('ford', 4292),
             ('opel', 3394),
             ('peugeot', 3361),
             ('fiat', 3256),
             ('renault', 2819)])

Premium German brands (Audi, Mercedes, BMW) are the most expensive ones by far. French brands (Peugeot, Renault) are the less expensive (with Fiat).  
It could be valuable to see if there is a link between those prices and the mileage of the cars. 

In [30]:
#Create empty dict for mean mileage
mean_mileage = {}

for brand in brands:
    #Filter dataframe by brands
    selected_rows = autos[autos['brand'] == brand]
    #Comupute rounded mean selling price
    mean_mileage[brand] = round(selected_rows['odometer_km'].mean())

In [31]:
#Convert dictionaries to series objects
mileage_series = pd.Series(mean_mileage)
price_series = pd.Series(price_wrt_brand)

In [32]:
#Create dataframe
df = pd.DataFrame(price_series, columns=["mean_price"])
df['mean_mileage'] = mileage_series

In [33]:
#Sort dataframe on prices
df.sort_values(["mean_price"], inplace=True, ascending=False)

In [34]:
df

,mean_price,mean_mileage
audi,9614,128910
mercedes_benz,8767,131019
bmw,8582,132866
volkswagen,5784,128235
seat,4811,120058
ford,4292,123495
opel,3394,128012
peugeot,3361,126073
fiat,3256,114416
renault,2819,126351


It seems that the price isn't strongly influenced by mileage and that the make itself, its "prestige" explains a lot of the final asking price.

## Keep On Cleaning

After all this data cleaning and a pre-analysis more could be done to improve data quality and clarity, one could for example
* **Data cleaning next steps** :
    * Identify categorical data that uses German words, translate them and map the values to their English counterparts
    * Check if missing values could be dealt with
    * Convert the dates to be uniform numeric data, so "2016-03-21" becomes the integer 20160321.
    * See if there are particular keywords in the name column that you can extract as new columns
* **Analysis next steps** :
    * Find the most common brand/model combinations
    * Split the odometer_km into groups, and use aggregation to see if average prices follows any patterns based on the mileage.
    * How much cheaper are cars with damage than their non-damaged counterparts?